In [1]:
import networkx as nx
import networkx.algorithms.community as nx_comm
import argparse
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch import optim

def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))

path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)


clu_res = 2 
clusters = nx_comm.louvain_communities(g, seed = 10, resolution = clu_res)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}

In [13]:

for node in g.nodes:
    cl = inverse_cluster_dict[node]
    cnt = sum([1 for neighbour in g[node] if inverse_cluster_dict[neighbour] == cl]) # in the same cluster
    g.nodes[node]['ai'] = cnt
    g.nodes[node]['bi'] =  g.degree[node] - cnt
    

C_edges = np.zeros((num_cluster, num_cluster), dtype=np.int64)

for edge in g.edges:
    ci = inverse_cluster_dict[edge[0]]
    cj = inverse_cluster_dict[edge[1]]
    C_edges[ci,cj] += 1
    C_edges[cj,ci] += 1
    


C_tensor = torch.tensor(C_edges, dtype = torch.float64)

In [12]:
eps = 1e-6
U = torch.tensor(np.load("optimized_cov/L_matrix_cov_res2_omega1_stf.npy"))
Z = torch.matmul(U, U.T)
V = torch.arcsin(Z/(1+eps))/(2*torch.pi)

Z[5:10, 5:10]

tensor([[ 1.0000,  0.1427, -1.0000,  0.6996,  0.7091],
        [ 0.1427,  1.0000, -0.1418,  0.3100,  0.2580],
        [-1.0000, -0.1418,  1.0000, -0.6975, -0.7071],
        [ 0.6996,  0.3100, -0.6975,  1.0000,  0.9982],
        [ 0.7091,  0.2580, -0.7071,  0.9982,  1.0000]], dtype=torch.float64)

In [15]:
C_tensor[5:10, 5:10]

tensor([[58512.,  3936.,  2332.,  3564.,  3661.],
        [ 3936., 32360.,  1699.,  5175.,  5990.],
        [ 2332.,  1699., 15314.,  3354.,  2987.],
        [ 3564.,  5175.,  3354., 21478.,  4299.],
        [ 3661.,  5990.,  2987.,  4299., 18708.]], dtype=torch.float64)